<a href="https://colab.research.google.com/github/AsliDK/Final-Year-Project/blob/main/fyp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Task 1: Environment Setup

Install Python: Ensure you have Python installed on your system.
Create a Virtual Environment: Use virtualenv to create an isolated environment for your project dependencies.
Install Required Libraries: Install Flask or Django, Tweepy, Pandas, GeoPandas, Scikit-learn, and any other necessary libraries using pip.

Task 2: Backend Development

Initialize Flask/Django Project: Create a new Flask or Django project as your application's backend framework.
Twitter API Integration:
Register your application with Twitter to obtain API keys.
Use Tweepy in a Python script to fetch geo-tagged tweets based on your criteria (location, keywords).

Data Processing:
Write Python scripts to clean and preprocess the data using Pandas.
Use GeoPandas for any spatial data operations needed.

Analysis & Machine Learning:
Implement algorithms to analyze patterns, preferences, and peak times in the collected data.
Use Scikit-learn for any machine learning models.

Task 3: Frontend Development

Basic Web Layout:
Create HTML templates for your main page, including forms for inputs (location, time frame, activity type).
Style your templates with CSS.

JavaScript and Map Integration:
Integrate Leaflet.js or Google Maps API to display an interactive map.
Use JavaScript to handle user inputs (forms) and interact with your backend for data retrieval.

Task 4: Connecting Backend and Frontend

API Development:
Develop RESTful APIs in your Flask/Django application to serve processed data to the frontend.

AJAX Requests:
Use AJAX in your JavaScript code to fetch data from your backend APIs and update the frontend dynamically.

Task 5: Data Visualization Integration
Integrate Plotly.js or D3.js
Add interactive charts and graphs to your frontend for displaying analysis results.

Task 6: Testing and Refinement
Unit Testing:
Write tests for both backend and frontend functionalities to ensure reliability.
System Integration Testing:
Test the entire system end-to-end, from data collection to visualization on the frontend.

Task 7: Deployment
Choose a Hosting Service: Decide on a cloud platform like Heroku, AWS, or Google Cloud.
Deploy Your Application: Follow the hosting service's guide to deploy your Flask/Django application.

Task 8: Documentation
Document Your Code: Ensure all your code is well-commented and documented.
User Guide: Write a user guide explaining how to use the system


Example Scenario:
A user selects "Soho, London" as the area of interest chooses "July 2023" as the time frame, and filters for "dining and nightlife" activities.

The system processes geo-tagged tweets from July 2023 related to dining and nightlife in Soho, identifies leisure hubs, and displays them on the map.

The map shows markers where activity is highest. Clicking on a marker displays detailed info such as popular venues, peak activity times, and possibly user sentiments or reviews extracted from the tweets.

The user can then generate a report summarising the findings, including recommendations for urban planning or business development in Soho.
This design bridges the gap between complex data analysis and user-friendly interaction, making urban planning insights accessible to a wider audience.


In [7]:
import requests

url = "https://api.foursquare.com/v3/places/search"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3PR9pOx+9xtJcWbhH2KCcLXqG7c83PTpavbRqXocj1Jc="
}

params = {
    "near": "London, UK",
    "limit": 50  # 100 mekan bilgisi almak için limiti belirle
}

response = requests.get(url, headers=headers, params=params)

print(response.text)


{"results":[{"fsq_id":"4ac518cdf964a520e6a520e3","categories":[{"id":10004,"name":"Art Gallery","short_name":"Art Gallery","plural_name":"Art Galleries","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/arts_entertainment/artgallery_","suffix":".png"}}],"chains":[],"closed_bucket":"VeryLikelyOpen","distance":193,"geocodes":{"main":{"latitude":51.508957,"longitude":-0.128678},"roof":{"latitude":51.508742,"longitude":-0.128459}},"link":"/v3/places/4ac518cdf964a520e6a520e3","location":{"address":"Trafalgar Sq","admin_region":"England","country":"GB","cross_street":"","formatted_address":"Trafalgar Sq, London, Greater London, WC2N 5DN","locality":"London","post_town":"London","postcode":"WC2N 5DN","region":"Greater London"},"name":"National Gallery","related_places":{"children":[{"fsq_id":"4cd183159d87224b4c405b3b","categories":[{"id":13035,"name":"Coffee Shop","short_name":"Coffee Shop","plural_name":"Coffee Shops","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffe

In [8]:
import pandas as pd
import json

# Assuming `response.text` contains the JSON response you provided
data = json.loads(response.text)

# Initialize an empty list to store the cleaned data
cleaned_data = []

# Iterate over each place in the results
for place in data['results']:
    # Extracting the place details
    name = place['name']
    categories = place['categories'][0]['name'] if place['categories'] else 'N/A'
    address = place['location']['formatted_address']
    latitude = place['geocodes']['main']['latitude']
    longitude = place['geocodes']['main']['longitude']

    # Append the data to the list as a tuple
    cleaned_data.append((name, categories, address, latitude, longitude))

# Convert the list of tuples into a DataFrame
df_places = pd.DataFrame(cleaned_data, columns=['Name', 'Category', 'Address', 'Latitude', 'Longitude'])

# Display the first few rows of the DataFrame
print(df_places.head(50))


                                Name                Category  \
0                   National Gallery             Art Gallery   
1                         WatchHouse             Coffee Shop   
2                     Lyceum Theatre            Concert Hall   
3                    Gielgud Theatre                     Bar   
4               Picturehouse Central           Movie Theater   
5                             Foyles               Bookstore   
6         International House London         Language School   
7          Big Ben (Elizabeth Tower)                Monument   
8              Palace of Westminster               Structure   
9                     British Museum          History Museum   
10                            Chanel          Clothing Store   
11                              Dior                Boutique   
12                        WatchHouse             Coffee Shop   
13                          Phillips             Art Gallery   
14              Mount Street Gardens    

In [9]:
def map_category_to_main(category):
    if 'Park' in category:
        return 'Park'
    elif 'Pub' in category:
        return 'Pub'
    elif any(store_keyword in category for store_keyword in ['Store', 'Shop', 'Market', 'Boutique', 'Mall']):
        return 'Store'
    elif any(restaurant_keyword in category for restaurant_keyword in ['Restaurant', 'Dining', 'Food', 'Cafe', 'Bakery', 'Bar', 'Bistro']):
        return 'Restaurant'
    else:
        return 'Other'

df_places['Main_Category'] = df_places['Category'].apply(map_category_to_main)


In [10]:
import folium

# Create a map centered around London
m = folium.Map(location=[51.5074, -0.1278], zoom_start=12)


In [41]:
category_colors = {
    'Park': 'green',
    'Pub': 'blue',
    'Store': 'red',
    'Restaurant': 'orange',
    'Structure': 'yellow',
    "History Museum": "darkpurple",
    'Other': 'purple'
}


In [42]:
for _, row in df_places.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Name']}: {row['Category']}",
        icon=folium.Icon(color=category_colors[row['Main_Category']])
    ).add_to(m)


In [43]:
for _, row in df_places.iterrows():
    popup_content = (
        f"Name: {row['Name']}<br>"
        f"Category: {row['Category']}<br>"
        f"Address: {row['Address']}<br>"

    )
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_content, max_width=300),  # Use Popup to allow for multi-line text
        icon=folium.Icon(color=category_colors[row['Main_Category']])
    ).add_to(m)

In [14]:
import folium

# Assuming df_places is already defined and includes 'Name', 'Category', 'Address', 'Timestamp'

# Create a map centered around London (or another relevant location)
m = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Define colors for each main category
category_colors = {
    'Park': 'green',
    'Pub': 'blue',
    'Store': 'red',
    'Restaurant': 'orange',
    'Structure': 'yellow',
    "History Museum": "darkpurple",
    'Other': 'purple'
}

# Add markers with customized popups
for _, row in df_places.iterrows():
    popup_content = (
        f"Name: {row['Name']}<br>"
        f"Category: {row['Category']}<br>"
        f"Address: {row['Address']}<br>"

    )
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_content, max_width=300),  # Use Popup to allow for multi-line text
        icon=folium.Icon(color=category_colors[row['Main_Category']])
    ).add_to(m)

# Display the map
m


In [50]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd
import folium


# Define the function to fetch and filter data
def fetch_and_display_data(area, activity_type):
    url = "https://api.foursquare.com/v3/places/search"
    headers = {
        "accept": "application/json",
        "Authorization": "fsq3PR9pOx+9xtJcWbhH2KCcLXqG7c83PTpavbRqXocj1Jc="
    }
    params = {
        "near": area,
        "limit": 50
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    cleaned_data = []
    for place in data['results']:
        name = place.get('name', 'N/A')
        categories = place['categories'][0]['name'] if place['categories'] else 'N/A'
        address = place.get('location', {}).get('formatted_address', 'N/A')
        latitude = place.get('geocodes', {}).get('main', {}).get('latitude', 0)
        longitude = place.get('geocodes', {}).get('main', {}).get('longitude', 0)

        if activity_type == 'All' or activity_type.lower() in categories.lower():
            cleaned_data.append((name, categories, address, latitude, longitude))

    df_places = pd.DataFrame(cleaned_data, columns=['Name', 'Category', 'Address', 'Latitude', 'Longitude'])

    # Create the map
    m = folium.Map(location=[51.5074, -0.1278], zoom_start=12)
    for _, row in df_places.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['Name']}: {row['Category']}",
            icon=folium.Icon(color='blue')
        ).add_to(m)

    # Display the map
    return m

# Create widgets for the user inputs
area_input = widgets.Text(value='London, UK', description='Area:', disabled=False)
activity_type_input = widgets.Dropdown(
    options=['All', 'Park', 'Pub', 'Store', 'Restaurant', 'Other'],
    value='All',
    description='Activity Type:',
    disabled=False,
)

# Button to submit the user input
submit_button = widgets.Button(description='Submit', disabled=False, button_style='', tooltip='Submit', icon='check')

   # Function to handle button click event
def on_submit_button_clicked(b):
    map_output.clear_output(wait=True)  # Clear the previous map output
    with map_output:
        area = area_input.value
        activity_type = activity_type_input.value

        # Fetch, filter, and display data
        map_display = fetch_and_display_data(area, activity_type)
        display(map_display)

# Create a separate output widget for the map
map_output = widgets.Output()

# Attach the event handler to the button
submit_button.on_click(on_submit_button_clicked)

# Display the widgets and the button
display(area_input, activity_type_input, submit_button, map_output)



Text(value='London, UK', description='Area:')

Dropdown(description='Activity Type:', options=('All', 'Park', 'Pub', 'Store', 'Restaurant', 'Other'), value='…

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit')

Output()

In [ ]:
import folium
import requests
import pandas as pd
import json

# Assuming df_places is already created and contains all your places data

# Create a mapping of category to marker color
category_colors = {
    "Art Gallery": "blue",
    "Coffee Shop": "green",
    "Concert Hall": "yellow",
    "Bar": "purple",
    "Movie Theater": "orange",
    "Bookstore": "darkblue",
    "Language School": "darkgreen",
    "Monument": "white",
    "Structure": "beige",
    "History Museum": "darkpurple",
    "Clothing Store": "lightgreen",
    "Boutique": "pink",
    # Add more categories and colors as needed
    "Urban Park": "lightred",
    "Grocery Store": "gray",
    "Cocktail Bar": "lightgray",
    "Bakery": "black",
    "Department Store": "lightblue",
    "Deli": "lightgreen",
    "Imported Food Store": "darkred",
    "Park": "green",
    "Restaurant": "",
    "Antique Store": "darkpink",
    "Jazz and Blues Venue": "orange",
    "Pub": "darkblue",
    "Landmarks and Outdoors": "darkgreen",
    "Zoo": "cadetblue",
    "Botanical Garden": "beige",
    "National Park": "darkpurple",
    "Hotel": "pink",
    "Garden": "lightred",
    "N/A": "lightgray"  # Default color for unspecified or unknown categories
}

# Create a base map centered around London
london_map = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Iterate over the rows in df_places to add categorized markers to the map
for index, row in df_places.iterrows():
    # Get the marker color from the category_colors mapping, default to 'gray' if category is not found
    marker_color = category_colors.get(row['Category'], 'gray')

    # Create and add the marker to the map
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Name']} - {row['Category']}\n{row['Address']}",
        icon=folium.Icon(color=marker_color, icon="info-sign")
    ).add_to(london_map)

# Display the map
london_map



<ipython-input-22-48c947c2c52f>:56: UserWarning: color argument of Icon should be one of: {'lightgray', 'green', 'lightblue', 'red', 'white', 'gray', 'pink', 'darkgreen', 'cadetblue', 'beige', 'purple', 'darkblue', 'lightgreen', 'blue', 'lightred', 'darkred', 'black', 'orange', 'darkpurple'}.
  icon=folium.Icon(color=marker_color, icon="info-sign")
